# Visualise the ``Data`` used in this project

In [ ]:
from typing import Tuple
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import lmfit
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy

from sdm_eurec4a import RepositoryPath, conversions
from sdm_eurec4a.identifications import (
    match_clouds_and_cloudcomposite,
    match_clouds_and_dropsondes,
    select_individual_cloud_by_id,
)
from sdm_eurec4a.input_processing.transfer import (
    PSD_LnNormal,
)
from sdm_eurec4a.input_processing import transfer

from sdm_eurec4a.reductions import shape_dim_as_dataarray
from sdm_eurec4a.visulization import (
    adjust_lightness_array,
    handler_map_alpha,
    set_custom_rcParams,
    plot_thermodynamics,
)

from importlib import reload

In [ ]:
reload(transfer)

<module 'sdm_eurec4a.input_processing.transfer' from '/home/m/m301096/repositories/sdm-eurec4a/src/sdm_eurec4a/input_processing/transfer.py'>

In [ ]:
def set_logxticks_meter(ax):
    xticks = [1e-6, 1e-3]
    xticklabels = [r"$10^{-6}$", r"$10^{-3}$"]
    ax.set_xticks(xticks, xticklabels)

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
dark_colors = adjust_lightness_array(default_colors, amount=0.5)

repo_path = RepositoryPath("levante")()
print(repo_path)

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = Path("/home/m/m301096/repositories/sdm-eurec4a/notebooks/presentation")
print(script_dir)


fig_dir = repo_path / "results" / script_dir.relative_to(repo_path) / "data"
print(fig_dir)

fig_dir.mkdir(parents=True, exist_ok=True)

/home/m/m301096/repositories/sdm-eurec4a
/home/m/m301096/repositories/sdm-eurec4a/notebooks/presentation
/home/m/m301096/repositories/sdm-eurec4a/results/notebooks/presentation/data


In [ ]:
identified_clouds_path = repo_path / Path(
    "data/observation/cloud_composite/processed/identified_clouds/identified_clusters_rain_mask_5.nc"
)
cloud_composite_path = repo_path / Path(
    f"data/observation/cloud_composite/processed/cloud_composite_si_units.nc"
)
drop_sondes_path = repo_path / Path("data/observation/dropsonde/processed/drop_sondes.nc")
distance_path = repo_path / Path(
    "data/observation/combined/distance_relations/distance_dropsondes_identified_clusters_rain_mask_5.nc"
)
satellite_path = repo_path / Path(
    "data/observation/satellite/samples/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20200261630156_BARBADOS-2KM-FD.level2.nc"
)

## Load datasets

In [ ]:
# %%

distance_IC_DS = xr.open_dataset(distance_path)

cloud_composite = xr.open_dataset(cloud_composite_path)

drop_sondes = xr.open_dataset(drop_sondes_path)

drop_sondes = drop_sondes.where(drop_sondes.alt <= 1600, drop=True)

# add relative humidity to the dataset

drop_sondes["relative_humidity"] = conversions.relative_humidity_from_tps(
    temperature=drop_sondes["air_temperature"],
    pressure=drop_sondes["pressure"],
    specific_humidity=drop_sondes["specific_humidity"],
)


# Add radii and altitude arrays to datasets

cloud_composite["radius_array"] = shape_dim_as_dataarray(
    da=cloud_composite["particle_size_distribution"], output_dim="radius"
)
cloud_composite["time_array"] = shape_dim_as_dataarray(
    da=cloud_composite["particle_size_distribution"], output_dim="time"
)
drop_sondes["alt_array"] = shape_dim_as_dataarray(da=drop_sondes["air_temperature"], output_dim="alt")

identified_clouds = xr.open_dataset(identified_clouds_path)
# select only clouds which are
# below 1500m and have a duration of at least 10s.
identified_clouds = identified_clouds.where(
    (identified_clouds["alt"] <= 1200)
    & (identified_clouds["alt"] >= 500)
    & (identified_clouds["vertical_extent"] <= 150)
    & (identified_clouds["duration"] >= np.timedelta64(3, "s"))
    # & (identified_clouds["liquid_water_content"] / identified_clouds["duration"].dt.seconds >=0.1)
    ,
    drop=True,
)

identified_clouds = xr.open_dataset(identified_clouds_path)
# select only clouds which are
# below 1500m and have a duration of at least 10s.
identified_clouds = identified_clouds.where(
    (identified_clouds["alt"] <= 1200)
    & (identified_clouds["alt"] >= 500)
    & (identified_clouds["vertical_extent"] <= 150)
    & (identified_clouds["duration"] >= np.timedelta64(3, "s"))
    # & (identified_clouds["liquid_water_content"] / identified_clouds["duration"].dt.seconds >=0.1)
    ,
    drop=True,
)

rename dataarray
rename dataarray
rename dataarray


In [ ]:
# Fit the ThermodynamicLinear models
thermo_fit = dict(
    air_temperature=transfer.ThermodynamicSplitLinear(),
    # specific_humidity=transfer.ThermodynamicSplitLinear(),
    # potential_temperature=transfer.ThermodynamicSplitLinear(),
    relative_humidity=transfer.ThermodynamicSplitLinear(),
)
for var in thermo_fit:
    transfer.fit_thermodynamics(
        da_thermo=drop_sondes.mean("time")[var],
        thermo_fit=thermo_fit[var],
        dim="alt",
    )

mean_x_split = np.mean([thermo_fit[key].get_x_split() for key in thermo_fit if key != "pressure"])

thermo_fit_all = dict(
    air_temperature=transfer.ThermodynamicSplitLinear(),
    specific_humidity=transfer.ThermodynamicSplitLinear(),
    potential_temperature=transfer.ThermodynamicSplitLinear(),
    relative_humidity=transfer.ThermodynamicSplitLinear(),
    pressure=transfer.ThermodynamicLinear(),
)

for var in thermo_fit_all:
    transfer.fit_thermodynamics(
        da_thermo=drop_sondes.mean("time")[var],
        thermo_fit=thermo_fit_all[var],
        dim="alt",
        x_split=mean_x_split,
    )


psd_fit_all = transfer.fit_2lnnormal_for_psd(
    da_psd=cloud_composite["particle_size_distribution"].mean("time"),
    dim="radius",
)

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=2, sharey=True, figsize=(10, 10))

fig, axs = plot_thermodynamics(
    fig=fig,
    axs=axs,
    drop_sondes=drop_sondes,
    fit_dict=thermo_fit_all,
)

for var, ax in zip(
    ["specific_humidity", "air_temperature", "relative_humidity", "potential_temperature"], axs.flatten()
):
    data = drop_sondes[var].mean("time")
    if var == "specific_humidity":
        data = data * 1000
    ax.plot(
        data,
        drop_sondes["alt"],
        color="k",
        label="mean",
    )
fig.tight_layout()

In [ ]:
print(thermo_fit_all["relative_humidity"])

nmodes = 1.00e+00
intercepts = [7.30e+01, ]
split x value = [3.52e+02, ]
slopes = [[ 0.03581309 -0.00680407], ]


In [ ]:
cc_cloud_base_time = cloud_composite.time.where(
    (cloud_composite["alt"] > 500) & (cloud_composite["alt"] <= 1200), drop=True
)
cc_cloud_base = cloud_composite.sel(time=cc_cloud_base_time)

In [ ]:
cc_match = match_clouds_and_cloudcomposite(identified_clouds, cloud_composite)

In [ ]:
max_spatial_distance = 100
max_temporal_distance = np.timedelta64(3, "h")
ds_match = match_clouds_and_dropsondes(
    ds_clouds=identified_clouds,
    ds_sonde=drop_sondes,
    ds_distance=distance_IC_DS,
    max_spatial_distance=max_spatial_distance,
    max_temporal_distance=max_temporal_distance,
)

In [ ]:
sat = xr.open_dataset(satellite_path)

X = sat["longitude"]
Y = sat["latitude"]
mask = np.isfinite(Y) & np.isfinite(X)

sat = sat.where(mask, drop=True)

## Plot functions

In [ ]:
# Draw a map of all dropsondes released during the campaign


def plot_map(
    fig: mpl.figure.Figure,
    ax,
    cloud_composite: xr.Dataset,
    drop_sondes: xr.Dataset,
    identified_clouds: xr.Dataset = None,
    extent: list = [-60, -56, 12, 15],
) -> Tuple:
    """
    Plots a map of the cloud composite and dropsondes and identified clouds.

    Parmeters

    """

    ax.coastlines()
    ax.add_feature(cfeature.LAND)
    # ax.add_feature(cfeature.OCEAN, color="navy", alpha=0.2)
    # ax.add_feature(cfeature.BORDERS, linestyle=":")
    gl = ax.gridlines(draw_labels=True, linestyle="--", alpha=0.5, color=[0.5, 0.5, 0.5])
    ax.set_extent(extent)

    ax.scatter(
        cloud_composite["lon"],
        cloud_composite["lat"],
        transform=ccrs.PlateCarree(),
        s=1,
        c=default_colors[0],
        marker=".",
        alpha=0.7,
        label="ATR",
        zorder=10,
    )

    ax.scatter(
        drop_sondes["flight_lon"],
        drop_sondes["flight_lat"],
        transform=ccrs.PlateCarree(),
        label="Drop sondes",
        color=dark_colors[1],
        marker="x",
        zorder=10,
        alpha=0.7,
    )

    if identified_clouds != None:
        ax.scatter(
            identified_clouds["lon"],
            identified_clouds["lat"],
            s=identified_clouds["horizontal_extent"] / 100,
            c="red",
            marker="o",
            alpha=0.5,
            # cmap = "Yellows_r",
            label="clouds",
            zorder=10,
        )

    return ax, gl


def plot_psd_pcolormesh_splitted(fig, ax, psd, split_radius: float = 4.5e-5):
    ds_lower = psd.sel(radius=slice(None, split_radius))
    ds_rain = psd.sel(radius=slice(split_radius, None))

    # plot cloud PSD distribution as pcolormesh

    vmin, vmax = 1, 1e8

    pc = ax.pcolormesh(
        ds_lower["radius"],
        ds_lower["time"],
        ds_lower.T,
        norm=mpl.colors.LogNorm(vmin, vmax),
        cmap="Blues",
        shading="nearest",
    )
    fig.colorbar(pc, ax=ax, label="Cloud and Drizzle drops #m$^{-3}$")
    pc = ax.pcolormesh(
        ds_rain["radius"],
        ds_rain["time"],
        ds_rain.T,
        norm=mpl.colors.LogNorm(vmin, 1e4),
        cmap="Reds",
        shading="nearest",
    )
    fig.colorbar(pc, ax=ax, label="Rain drops in #m$^{-3}$")
    ax.set_xscale("log")
    ax.set_xlabel("Radius in m")
    ax.set_ylabel("Time")


def plot_psd_scatter(fig, ax, ds_cloudcomposite, split_radius=4.5e-5):
    ds_lower = ds_cloudcomposite.sel(radius=slice(None, split_radius))
    ds_rain = ds_cloudcomposite.sel(radius=slice(split_radius, None))

    style = dict(
        marker="o",
        edgecolor="none",
        s=10,
        alpha=0.5,
    )

    ax.scatter(
        ds_rain.radius_array,
        ds_rain.particle_size_distribution,
        c=default_colors[0],
        label="rain drops",
        **style,
    )

    ax.scatter(
        ds_lower.radius_array,
        ds_lower.particle_size_distribution,
        c=default_colors[1],
        label="cloud and drizzle",
        **style,
    )

    ax.legend(handler_map=handler_map_alpha(), loc="upper right")

    ax.set_xlabel(r"Radius $m$")
    ax.set_ylabel(r"Occurence  $\#/m^{-3}$")
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlim(1e-7, 4e-3)
    ax.set_ylim(1e0, 1e9)

    ax.set_yticklabels([1e0, 1e6])
    ax.set_xticklabels([1e-6, 1e-3])


def plot_msd_scatter(fig, ax, ds_cloudcomposite, split_radius=4.5e-5):
    ds_lower = ds_cloudcomposite.sel(radius=slice(None, split_radius))
    ds_rain = ds_cloudcomposite.sel(radius=slice(split_radius, None))

    style = dict(
        marker="o",
        edgecolor="none",
        s=10,
        alpha=0.5,
    )

    ax.set_ylim(1e-10, 2e-4)
    ax.set_yscale("log")
    ax.set_xscale("log")

    ax.scatter(
        ds_rain.radius_array,
        1e3 * ds_rain.mass_size_distribution,
        c=default_colors[0],
        label="rain drops",
        **style,
    )

    ax.scatter(
        ds_lower.radius_array,
        1e3 * ds_lower.mass_size_distribution,
        c=default_colors[1],
        label="cloud and drizzle",
        **style,
    )

    ax.legend(handler_map=handler_map_alpha(), loc="lower right")

    ax.set_xlabel(r"radius in $m$")
    ax.set_ylabel(r"Mass in $g m^{-3}$")
    ax.set_xlim(1e-7, 4e-3)
    ax.set_ylim(1e-10, 1e-4)
    ax.set_yticklabels([1e-9, 1e-5])
    ax.set_xticklabels([1e-6, 1e-3])


# def plot_thermodynamics(fig, ds_dropsondes, ds_cloud):
#     axs = fig.subplots(ncols=2, nrows=2, sharey=True)
#     scatter_style = dict(alpha=0.75, linewidth=0.7)

#     ax_q, ax_ta = axs[0]
#     ax_rh, ax_theta = axs[1]
#     ax_q.set_xlim(0, 20)
#     ax_rh.set_xlim(30, 110)
#     ax_ta.set_xlim(285, 305)
#     ax_theta.set_xlim(295, 308)

#     # First plot
#     ax_q.plot(
#         1e3 * ds_dropsondes["specific_humidity"].T,
#         ds_dropsondes["alt"].T,
#         color=default_colors[0],
#         # label="Spec. Hum.",
#         **scatter_style,
#     )

#     ax_q.set_xlabel("Specific humidity [g/kg]")
#     ax_q.set_ylabel("alt [m]")
#     ax_q.xaxis.label.set_color(default_colors[0])  # Set the color of x-axis label
#     ax_q.tick_params(axis="x", colors=default_colors[0])  # Set the color of x-axis ticks

#     # First plot
#     ax_rh.plot(
#         ds_dropsondes["relative_humidity"].T,
#         ds_dropsondes["alt"].T,
#         color=default_colors[3],
#         # label="Spec. Hum.",
#         **scatter_style,
#     )
#     ax_rh.axvline(100, color="grey", linestyle="-", label="Saturation")
#     ax_rh.set_xlabel("Relative humidity [%]")
#     ax_rh.set_ylabel("alt [m]")
#     ax_rh.xaxis.label.set_color(default_colors[3])  # Set the color of x-axis label
#     ax_rh.tick_params(axis="x", colors=default_colors[3])  # Set the color of x-axis ticks

#     # Second plot on a new x-axis
#     ax_ta.plot(
#         ds_dropsondes["air_temperature"].T,
#         ds_dropsondes["alt"].T,
#         color=default_colors[1],
#         # label="Air Temperature",
#         **scatter_style,
#     )

#     ax_ta.set_xlabel("Air Temperature [K]")
#     ax_ta.xaxis.label.set_color(default_colors[1])  # Set the color of x-axis label
#     ax_ta.tick_params(axis="x", colors=default_colors[1])  # Set the color of x-axis ticks

#     # Thrid plot on a new x-axis
#     ax_theta.plot(
#         ds_dropsondes["potential_temperature"].T,
#         ds_dropsondes["alt"].T,
#         color=default_colors[2],
#         # label="Pot. Temperature",
#         **scatter_style,
#     )

#     ax_theta.set_xlabel("Pot. Temperature [K]")
#     ax_theta.xaxis.label.set_color(default_colors[2])  # Set the color of x-axis label
#     ax_theta.tick_params(axis="x", colors=default_colors[2])  # Set the color of x-axis ticks

#     for ax in axs.flatten():
#         ax.set_ylim(0, 1500)
#         ax.axhline(ds_cloud.alt, color="r", linestyle="--", label="PSD measurement")
#         ax.legend(handler_map=handler_map_alpha())
#         ax.tick_params(axis="x", labelrotation=-33)

In [ ]:
individual_cloud = select_individual_cloud_by_id(identified_clouds, 301)
cc_individual = match_clouds_and_cloudcomposite(individual_cloud, cloud_composite)
max_spatial_distance = 100
max_temporal_distance = np.timedelta64(3, "h")
ds_individual = match_clouds_and_dropsondes(
    ds_clouds=individual_cloud,
    ds_sonde=drop_sondes,
    ds_distance=distance_IC_DS,
    max_spatial_distance=max_spatial_distance,
    max_temporal_distance=max_temporal_distance,
)

### Map of observation data

In [ ]:
fig, axs = plt.subplots(
    ncols=2,
    nrows=2,
    subplot_kw={"projection": ccrs.PlateCarree()},
    figsize=(21, 12),
)


setups = dict(
    all=dict(
        data=dict(
            drop_sondes=drop_sondes,
            cloud_composite=cc_cloud_base,
        ),
        title="Drop sonde releases and ATR measurements at cloud base",
        ax=axs[0, 0],
    ),
    match=dict(
        data=dict(
            drop_sondes=ds_match,
            cloud_composite=cc_match,
            identified_clouds=identified_clouds,
        ),
        title="Related to clouds along ATR flight path\nDrop sonde releases and ATR measurements at cloud base",
        ax=axs[0, 1],
    ),
    individual=dict(
        data=dict(
            drop_sondes=ds_individual,
            cloud_composite=cc_individual,
            identified_clouds=individual_cloud,
        ),
        title=f"Drop sonde releases and ATR measurements for example cloud\n{individual_cloud.time.dt.strftime('%Y-%m-%d %H:%M')[0].values}",
        ax=axs[1, 0],
    ),
    individual_satellite=dict(
        data=dict(
            drop_sondes=ds_individual,
            cloud_composite=cc_individual,
            identified_clouds=individual_cloud,
        ),
        title=f"Drop sonde releases and ATR measurements for example cloud\n{individual_cloud.time.dt.strftime('%Y-%m-%d %H:%M')[0].values}",
        ax=axs[1, 1],
    ),
)

for key in setups:
    data_dict = setups[key]["data"]
    ax = setups[key]["ax"]
    ax, gl = plot_map(
        fig=fig,
        ax=ax,
        **data_dict,
    )
    gl.right_labels = False
    gl.top_labels = False

    lgnd = ax.legend()
    for handle in lgnd.legend_handles:
        handle.set_sizes([100.0])
    ax.spines["geo"].set_visible(False)
    ax.set_title(setups[key]["title"])

for key in ["individual_satellite"]:
    ax = setups[key]["ax"]
    ax.pcolormesh(
        sat["longitude"],
        sat["latitude"],
        sat["refl_0_47um_nom"],
        cmap="Blues_r",
        zorder=1,
        vmin=sat["refl_0_47um_nom"].min()
        - 0.5 * (sat["refl_0_47um_nom"].max() - sat["refl_0_47um_nom"].min()),
        vmax=sat["refl_0_47um_nom"].max()
        - 0.5 * (sat["refl_0_47um_nom"].max() - sat["refl_0_47um_nom"].min()),
    )

fig.tight_layout()
fig.savefig(fig_dir / "sonde_art_locations.png", dpi=500)

In [ ]:
setups = dict(
    all=dict(
        data=dict(
            drop_sondes=drop_sondes,
            cloud_composite=cc_cloud_base,
        ),
        title="Drop sonde releases and ATR measurements at cloud base",
    ),
    match=dict(
        data=dict(
            drop_sondes=ds_match,
            cloud_composite=cc_match,
            identified_clouds=identified_clouds,
        ),
        title="Related to clouds along ATR flight path\nDrop sonde releases and ATR measurements at cloud base",
    ),
    individual=dict(
        data=dict(
            drop_sondes=ds_individual,
            cloud_composite=cc_individual,
            identified_clouds=individual_cloud,
        ),
        title=f"Drop sonde releases and ATR measurements for example cloud.",  # \n{individual_cloud.time.dt.strftime('%Y-%m-%d %H:%M')[0].values}",
    ),
    individual_satellite=dict(
        data=dict(
            drop_sondes=ds_individual,
            cloud_composite=cc_individual,
            identified_clouds=individual_cloud,
        ),
        title=f"Drop sonde releases and ATR measurements for example cloud",  # \n{individual_cloud.time.dt.strftime('%Y-%m-%d %H:%M')[0].values}",
    ),
)

for key in setups:
    fig, ax = plt.subplots(
        ncols=1,
        nrows=1,
        subplot_kw={"projection": ccrs.PlateCarree()},
        figsize=(10, 6),
        layout="constrained",
    )
    setups[key]["ax"] = ax
    setups[key]["fig"] = fig

for key in setups:
    data_dict = setups[key]["data"]
    ax = setups[key]["ax"]
    ax, gl = plot_map(
        fig=fig,
        ax=ax,
        **data_dict,
    )
    gl.right_labels = False
    gl.top_labels = False

    lgnd = ax.legend()
    for handle in lgnd.legend_handles:
        handle.set_sizes([100.0])
    ax.spines["geo"].set_visible(False)
    ax.set_title(setups[key]["title"])

for key in ["individual_satellite"]:
    ax = setups[key]["ax"]
    ax.pcolormesh(
        sat["longitude"],
        sat["latitude"],
        sat["refl_0_47um_nom"],
        cmap="Blues_r",
        zorder=1,
        vmin=sat["refl_0_47um_nom"].min()
        - 0.5 * (sat["refl_0_47um_nom"].max() - sat["refl_0_47um_nom"].min()),
        vmax=sat["refl_0_47um_nom"].max()
        - 0.5 * (sat["refl_0_47um_nom"].max() - sat["refl_0_47um_nom"].min()),
    )

for key in setups:
    fig = setups[key]["fig"]
    fig.set_size_inches(10, 6, forward=True)
    # fig.tight_layout()
    fig.savefig(fig_dir / f"sonde_art_locations_{key}.png", dpi=500)

### Random ATR sample

In [ ]:
np.random.seed(42)
t_random = np.random.choice(cloud_composite.time.shape[0], 1, replace=False)
random_cc = cloud_composite.isel(time=t_random)
fig, axs = plt.subplots(ncols=2)
plot_psd_scatter(fig=fig, ax=axs[0], ds_cloudcomposite=random_cc, split_radius=4.5e-5)
plot_msd_scatter(fig=fig, ax=axs[1], ds_cloudcomposite=random_cc, split_radius=4.5e-5)

N_total = random_cc["particle_size_distribution"].sum().values
N_mean = random_cc["particle_size_distribution"].sum("radius").mean("time").values

M_total = random_cc["mass_size_distribution"].sum().values * 1e3
M_mean = random_cc["mass_size_distribution"].sum("radius").mean("time").values * 1e3

axs[0].set_title(
    "Particle size distribution"
    # + "\n"
    # + r"$N_{total}= $"
    # + f"{N_total:.2e}"
    # + r"  $N_{mean}= $"
    # + f"{N_mean:.2e}",
    ,
    fontsize=10,
)
axs[1].set_title(
    "Mass size distribution"
    # + "\n"
    # + r"$M_{total}= $"
    # + f"{M_total:.3f}"
    # + r"$g m^{-3}$"
    # + r"  $M_{mean}= $"
    # + f"{M_mean:.2e}"
    # + r"$g m^{-3}$",
    ,
    fontsize=10,
)

fig.tight_layout()

fig.savefig(fig_dir / "psd_random.png", dpi=500)

### Specific cloud thermodynmaics and PSD

In [ ]:
individual_cloud = select_individual_cloud_by_id(identified_clouds, 301)
cc_individual = match_clouds_and_cloudcomposite(individual_cloud, cloud_composite)
max_spatial_distance = 100
max_temporal_distance = np.timedelta64(3, "h")
ds_individual = match_clouds_and_dropsondes(
    ds_clouds=individual_cloud,
    ds_sonde=drop_sondes,
    ds_distance=distance_IC_DS,
    max_spatial_distance=max_spatial_distance,
    max_temporal_distance=max_temporal_distance,
)

#### Fitting thermodynamics and PSD for individual cloud

In [ ]:
# Fit the ThermodynamicLinear models
thermo_fit = dict(
    # air_temperature=transfer.ThermodynamicSplitLinear(),
    # specific_humidity=transfer.ThermodynamicSplitLinear(),
    # potential_temperature=transfer.ThermodynamicSplitLinear(),
    relative_humidity=transfer.ThermodynamicSplitLinear(),
)
for var in thermo_fit:
    transfer.fit_thermodynamics(
        da_thermo=ds_individual[var],
        thermo_fit=thermo_fit[var],
        dim="alt",
    )

mean_x_split = np.mean([thermo_fit[key].get_x_split() for key in thermo_fit if key != "pressure"])

thermo_fit_individual = dict(
    air_temperature=transfer.ThermodynamicSplitLinear(),
    specific_humidity=transfer.ThermodynamicSplitLinear(),
    potential_temperature=transfer.ThermodynamicSplitLinear(),
    relative_humidity=transfer.ThermodynamicSplitLinear(),
    pressure=transfer.ThermodynamicLinear(),
)

for var in thermo_fit_individual:
    transfer.fit_thermodynamics(
        da_thermo=ds_individual[var],
        thermo_fit=thermo_fit_individual[var],
        dim="alt",
        x_split=mean_x_split,
    )


psd_fit_individual = transfer.fit_2lnnormal_for_psd(
    da_psd=cc_individual["particle_size_distribution"].where(
        cc_individual["particle_size_distribution"] != 0
    ),
    dim="radius",
)

#### PSD visulaise

# mean cloud property

In [ ]:
# Draw a map of all dropsondes released during the campaign


def plot_map(
    fig: mpl.figure.Figure,
    ax,
    cloud_composite: xr.Dataset,
    drop_sondes: xr.Dataset,
    identified_clouds: xr.Dataset = None,
    extent: list = [-60, -56, 12, 15],
) -> Tuple:
    """
    Plots a map of the cloud composite and dropsondes and identified clouds.

    Parmeters

    """

    ax.coastlines()
    ax.add_feature(cfeature.LAND)
    # ax.add_feature(cfeature.OCEAN, color="navy", alpha=0.2)
    # ax.add_feature(cfeature.BORDERS, linestyle=":")
    gl = ax.gridlines(draw_labels=True, linestyle="--", alpha=0.5, color=[0.5, 0.5, 0.5])
    ax.set_extent(extent)

    ax.scatter(
        cloud_composite["lon"],
        cloud_composite["lat"],
        transform=ccrs.PlateCarree(),
        s=1,
        c=default_colors[0],
        marker=".",
        alpha=0.7,
        label="ATR",
        zorder=10,
    )

    ax.scatter(
        drop_sondes["flight_lon"],
        drop_sondes["flight_lat"],
        transform=ccrs.PlateCarree(),
        label="Drop sondes",
        color=dark_colors[1],
        marker="x",
        zorder=10,
        alpha=0.7,
    )

    if identified_clouds != None:
        ax.scatter(
            identified_clouds["lon"],
            identified_clouds["lat"],
            s=identified_clouds["horizontal_extent"] / 100,
            c="red",
            marker="o",
            alpha=0.5,
            # cmap = "Yellows_r",
            label="clouds",
            zorder=10,
        )

    return ax, gl


def plot_psd_pcolormesh_splitted(fig, ax, psd, split_radius: float = 4.5e-5):
    ds_lower = psd.sel(radius=slice(None, split_radius))
    ds_rain = psd.sel(radius=slice(split_radius, None))

    # plot cloud PSD distribution as pcolormesh

    vmin, vmax = 1, 1e8

    pc = ax.pcolormesh(
        ds_lower["radius"],
        ds_lower["time"],
        ds_lower.T,
        norm=mpl.colors.LogNorm(vmin, vmax),
        cmap="Blues",
        shading="nearest",
    )
    fig.colorbar(pc, ax=ax, label="Cloud and Drizzle drops #m$^{-3}$")
    pc = ax.pcolormesh(
        ds_rain["radius"],
        ds_rain["time"],
        ds_rain.T,
        norm=mpl.colors.LogNorm(vmin, 1e4),
        cmap="Reds",
        shading="nearest",
    )
    fig.colorbar(pc, ax=ax, label="Rain drops in #m$^{-3}$")
    ax.set_xscale("log")
    ax.set_xlabel("Radius in m")
    ax.set_ylabel("Time")


def plot_psd_scatter(fig, ax, ds_cloudcomposite, split_radius=4.5e-5):
    ds_lower = ds_cloudcomposite.sel(radius=slice(None, split_radius))
    ds_rain = ds_cloudcomposite.sel(radius=slice(split_radius, None))

    style = dict(
        marker="o",
        edgecolor="none",
        s=10,
        alpha=0.5,
    )

    ax.scatter(
        ds_rain.radius_array,
        ds_rain.particle_size_distribution,
        c=default_colors[0],
        label="rain drops",
        **style,
    )

    ax.scatter(
        ds_lower.radius_array,
        ds_lower.particle_size_distribution,
        c=default_colors[1],
        label="cloud and drizzle",
        **style,
    )

    ax.legend(handler_map=handler_map_alpha(), loc="upper right")

    ax.set_xlabel(r"Radius $m$")
    ax.set_ylabel(r"Occurence  $\#/m^{-3}$")
    ax.set_xscale("log")
    ax.set_yscale("log")

    xticks = [1e-6, 1e-3]
    xticklabels = [r"$10^{-6}$", r"$10^{-3}$"]
    yticks = [1e0, 1e6]
    yticklabels = [r"$10^0$", r"$10^6$"]
    ax.set_xticks(xticks, xticklabels)
    ax.set_yticks(yticks, yticklabels)
    ax.set_xlim(1e-7, 4e-3)
    ax.set_ylim(1e0, 1e9)


def plot_msd_scatter(fig, ax, ds_cloudcomposite, split_radius=4.5e-5):
    ds_lower = ds_cloudcomposite.sel(radius=slice(None, split_radius))
    ds_rain = ds_cloudcomposite.sel(radius=slice(split_radius, None))

    style = dict(
        marker="o",
        edgecolor="none",
        s=10,
        alpha=0.5,
    )

    ax.set_ylim(1e-10, 2e-4)
    ax.set_yscale("log")
    ax.set_xscale("log")

    ax.scatter(
        ds_rain.radius_array,
        1e3 * ds_rain.mass_size_distribution,
        c=default_colors[0],
        label="rain drops",
        **style,
    )

    ax.scatter(
        ds_lower.radius_array,
        1e3 * ds_lower.mass_size_distribution,
        c=default_colors[1],
        label="cloud and drizzle",
        **style,
    )

    ax.legend(handler_map=handler_map_alpha(), loc="lower right")

    ax.set_xlabel(r"radius in $m$")
    ax.set_ylabel(r"Mass in $g m^{-3}$")
    ax.set_xlim(1e-7, 4e-3)
    ax.set_ylim(1e-10, 1e-4)
    ax.set_yticklabels([1e-9, 1e-5])
    ax.set_xticklabels([1e-6, 1e-3])


# def plot_thermodynamics(fig, ds_dropsondes, ds_cloud):
#     axs = fig.subplots(ncols=2, nrows=2, sharey=True)
#     scatter_style = dict(alpha=0.75, linewidth=0.7)

#     ax_q, ax_ta = axs[0]
#     ax_rh, ax_theta = axs[1]
#     ax_q.set_xlim(0, 20)
#     ax_rh.set_xlim(30, 110)
#     ax_ta.set_xlim(285, 305)
#     ax_theta.set_xlim(295, 308)

#     # First plot
#     ax_q.plot(
#         1e3 * ds_dropsondes["specific_humidity"].T,
#         ds_dropsondes["alt"].T,
#         color=default_colors[0],
#         # label="Spec. Hum.",
#         **scatter_style,
#     )

#     ax_q.set_xlabel("Specific humidity [g/kg]")
#     ax_q.set_ylabel("alt [m]")
#     ax_q.xaxis.label.set_color(default_colors[0])  # Set the color of x-axis label
#     ax_q.tick_params(axis="x", colors=default_colors[0])  # Set the color of x-axis ticks

#     # First plot
#     ax_rh.plot(
#         ds_dropsondes["relative_humidity"].T,
#         ds_dropsondes["alt"].T,
#         color=default_colors[3],
#         # label="Spec. Hum.",
#         **scatter_style,
#     )
#     ax_rh.axvline(100, color="grey", linestyle="-", label="Saturation")
#     ax_rh.set_xlabel("Relative humidity [%]")
#     ax_rh.set_ylabel("alt [m]")
#     ax_rh.xaxis.label.set_color(default_colors[3])  # Set the color of x-axis label
#     ax_rh.tick_params(axis="x", colors=default_colors[3])  # Set the color of x-axis ticks

#     # Second plot on a new x-axis
#     ax_ta.plot(
#         ds_dropsondes["air_temperature"].T,
#         ds_dropsondes["alt"].T,
#         color=default_colors[1],
#         # label="Air Temperature",
#         **scatter_style,
#     )

#     ax_ta.set_xlabel("Air Temperature [K]")
#     ax_ta.xaxis.label.set_color(default_colors[1])  # Set the color of x-axis label
#     ax_ta.tick_params(axis="x", colors=default_colors[1])  # Set the color of x-axis ticks

#     # Thrid plot on a new x-axis
#     ax_theta.plot(
#         ds_dropsondes["potential_temperature"].T,
#         ds_dropsondes["alt"].T,
#         color=default_colors[2],
#         # label="Pot. Temperature",
#         **scatter_style,
#     )

#     ax_theta.set_xlabel("Pot. Temperature [K]")
#     ax_theta.xaxis.label.set_color(default_colors[2])  # Set the color of x-axis label
#     ax_theta.tick_params(axis="x", colors=default_colors[2])  # Set the color of x-axis ticks

#     for ax in axs.flatten():
#         ax.set_ylim(0, 1500)
#         ax.axhline(ds_cloud.alt, color="r", linestyle="--", label="PSD measurement")
#         ax.legend(handler_map=handler_map_alpha())
#         ax.tick_params(axis="x", labelrotation=-33)

In [ ]:
fig, axs = plt.subplots(ncols=2)
cc = cc_individual
cc = cc.where(cc["particle_size_distribution"] != 0)

cc["particle_size_distribution"] = cc["particle_size_distribution"] * 0 + (
    cc["particle_size_distribution"].median("time")
)
cc["mass_size_distribution"] = conversions.msd_from_psd(ds=cc, psd_name="particle_size_distribution")
plot_psd_scatter(fig=fig, ax=axs[0], ds_cloudcomposite=cc, split_radius=4.5e-5)
plot_msd_scatter(fig=fig, ax=axs[1], ds_cloudcomposite=cc, split_radius=4.5e-5)

# psd_fit_individual_mean = transfer.fit_2lnnormal_for_psd(
#     da_psd=cc["particle_size_distribution"],
#     dim="radius",
# )

N_total = cc["particle_size_distribution"].sum().values
N_mean = cc["particle_size_distribution"].sum("radius").mean("time").values
# N_mean = cc["particle_size_distribution"].sel(radius = slice(45e-6, None)).sum("radius").mean("time").values

M_total = cc["mass_size_distribution"].sum().values * 1e3
M_mean = cc["mass_size_distribution"].sum("radius").mean("time").values * 1e3

axs[0].set_title(
    "Particle size distribution\n"
    + r"$N_{total}= $"
    + f"{N_total:.2e}"
    + r"$ m^{-3}$"
    + r"  $N_{mean}= $"
    + f"{N_mean:.2e}",
    fontsize=10,
)


# axs[0].plot(
#     cc.radius,
#     psd_fit_individual_mean.eval_func(cloud_composite.radius),
#     label="rain fit",
#     linestyle="-",
#     color="k",
#     alpha = 0.75,
# )

axs[1].set_title(
    "Mass size distribution\n"
    + r"$M_{total}= $"
    + f"{M_total:.2f} "
    + r"$ g {m^{-3}}$"
    + r"  $M_{mean}= $"
    + f"{M_mean:.2f} "
    + r"$ g {m^{-3}}$",
    fontsize=10,
)

fig.suptitle("PSD for example cloud", fontsize=12)
fig.tight_layout()
fig.savefig(fig_dir / "psd_cloud_mean.png", dpi=500)

/tmp/ipykernel_776295/3720145580.py:176: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([1e-9, 1e-5])
/tmp/ipykernel_776295/3720145580.py:177: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([1e-6, 1e-3])


In [ ]:
fig, axs = plt.subplots(ncols=2)
cc = cc_individual
cc = cc.where(cc["particle_size_distribution"] != 0)

plot_psd_scatter(fig=fig, ax=axs[0], ds_cloudcomposite=cc, split_radius=4.5e-5)
plot_msd_scatter(fig=fig, ax=axs[1], ds_cloudcomposite=cc, split_radius=4.5e-5)


axs[0].plot(
    cc.radius,
    psd_fit_individual.eval_func(cloud_composite.radius),
    label="rain fit",
    linestyle="-",
    color="k",
    alpha=0.75,
)


# psd_fit = transfer.fit_2lnnormal_for_psd(
#     da_psd=cc["particle_size_distribution"],
#     dim="radius",
# )

N_total = cc["particle_size_distribution"].sum().values
N_mean = cc["particle_size_distribution"].sum("radius").mean("time").values
# N_mean = cc["particle_size_distribution"].sel(radius = slice(45e-6, None)).sum("radius").mean("time").values

M_total = cc["mass_size_distribution"].sum().values * 1e3
M_mean = cc["mass_size_distribution"].sum("radius").mean("time").values * 1e3

axs[0].set_title(
    "Particle size distribution\n"
    + r"$N_{total}= $"
    + f"{N_total:.2e}"
    + r"$ m^{-3}$"
    + r"  $N_{mean}= $"
    + f"{N_mean:.2e}",
    fontsize=10,
)


axs[1].set_title(
    "Mass size distribution\n"
    + r"$M_{total}= $"
    + f"{M_total:.2f} "
    + r"$ g {m^{-3}}$"
    + r"  $M_{mean}= $"
    + f"{M_mean:.2f} "
    + r"$ g {m^{-3}}$",
    fontsize=10,
)


fig.tight_layout()
fig.savefig(fig_dir / "psd_cloud.png", dpi=500)

In [ ]:
cc_fitted["particle_size_distribution"] = psd_fit_individual_mean.eval_func(
    cc_individual["radius_array"]
)
cc_fitted["mass_size_distribution"] = conversions.msd_from_psd(cc_fitted)

In [ ]:
fig, axs = plt.subplots(ncols=2)
cc = cc_fitted
cc = cc.where(cc["particle_size_distribution"] != 0)
cc["particle_size_distribution"] = cc["particle_size_distribution"]
plot_psd_scatter(fig=fig, ax=axs[0], ds_cloudcomposite=cc, split_radius=4.5e-5)
plot_msd_scatter(fig=fig, ax=axs[1], ds_cloudcomposite=cc, split_radius=4.5e-5)

N_total = cc["particle_size_distribution"].sum().values
N_mean = cc["particle_size_distribution"].sum("radius").mean("time").values
# N_mean = cc["particle_size_distribution"].sel(radius = slice(45e-6, None)).sum("radius").mean("time").values

M_total = cc["mass_size_distribution"].sum().values * 1e3
M_mean = cc["mass_size_distribution"].sum("radius").mean("time").values * 1e3

axs[0].set_title(
    "Particle size distribution\n"
    + r"$N_{total}= $"
    + f"{N_total:.2e}"
    + r"$ m^{-3}$"
    + r"  $N_{mean}= $"
    + f"{N_mean:.2e}",
    fontsize=10,
)
axs[1].set_title(
    "Mass size distribution\n"
    + r"$M_{total}= $"
    + f"{M_total:.2f} "
    + r"$ g {m^{-3}}$"
    + r"  $M_{mean}= $"
    + f"{M_mean:.2f} "
    + r"$ g {m^{-3}}$",
    fontsize=10,
)


fig.tight_layout()
fig.savefig(fig_dir / "psd_cloud.png", dpi=500)

In [ ]:
fig, ax = plt.subplots()
plot_psd_pcolormesh_splitted(fig, ax, cc_individual["particle_size_distribution"], split_radius=4.5e-5)

fig.savefig(fig_dir / "psd_cloud_pcolormesh.png", dpi=500)


fig, ax = plt.subplots()
plot_psd_pcolormesh_splitted(fig, ax, psd_fit.eval_func(cc_individual["radius_array"]))

fig.savefig(fig_dir / "psd_cloud_pcolormesh_fit.png", dpi=500)

In [ ]:
fig = plt.figure(figsize=(12, 9))
axs = fig.subplots(ncols=2, nrows=2, sharey=True)

plot_thermodynamics(
    fig=fig,
    axs=axs,
    drop_sondes=ds_individual,
    fit_dict=thermo_fit_individual,
)

for ax in axs.flatten():
    ax.axhline(individual_cloud["alt"], color="r", linestyle="--", label="PSD measurement")
    ax.legend(handler_map=handler_map_alpha())
    ax.tick_params(axis="x", labelrotation=-33)

fig.suptitle(
    f"Thermodynamic profiles related to individual cloud\n{individual_cloud.time.dt.strftime('%Y-%m-%d %H:%M')[0].values}",
    fontsize=16,
)
fig.tight_layout()
fig.savefig(fig_dir / "thermo_individual.png", dpi=500)